In [1]:
calc_type = 'Standard' # 'Standard' or 'Large'

In [2]:
if calc_type == 'Standard':
    old_rate = '9-I/III'
    new_rate = '9-IV'
elif calc_type == 'Large':
    old_rate = '9-II'
    new_rate = '9-V'

In [3]:
import pandas as pd 
import numpy as np 
import pyodbc
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, accuracy_score
from sklearn.model_selection import train_test_split
import seaborn as sns
import statsmodels.api as sm
import ast
from IPython.display import clear_output
from datetime import datetime

In [4]:
def string_to_list(s):
    try:
        result = ast.literal_eval(s)
        if isinstance(result, dict):
            return result
        else:
            raise ValueError("The string does not represent a list.")
    except Exception as e:
        print(f"Error: {e}")
        return None

In [5]:
conn1 = pyodbc.connect('Driver={SQL Server};'
                              'Server=UTIL-PROD-DB;'
                              'Database=NewClientInfo;')

In [6]:
def Acc(Anumbers, conn1):
    '''Takes account number, not account ID'''

    sql1 = f"""SELECT distinct Acc.[AccountID] as AccountID
                                ,Acc.[AccountNumber] as AccountNumber
                    ,Client.[CompanyName] as Client
                FROM [NewClientInfo].[dbo].[Accounts] Acc 
                JOIN [NewClientInfo].[dbo].[Client] on [NewClientInfo].[dbo].[Client].[ClientID]=Acc.[AccountClientID]
                WHERE  Acc.[AccountID] IN ({','.join(Anumbers)})
"""
    bills = pd.read_sql(sql1, conn1)
    
    return bills

In [7]:
sql1 = r"""SELECT 'PDF Location' = 'Q:\_UTILISAVE LLC\UtilityBilling\Production\100001\' + UtilImp.[Url]
,A.AccountID
,RawData
,EGOS.Usage
,EGOS.Demand
,DateFrom
,DateTo
,BillAmount
,EGOS.StatementDate
,ami.AMIUsage / ami.Usage AS 'Usage Disc'


  FROM [NewClientInfo].[dbo].[EGOS_BillingDetails] AS EGOS
  JOIN [ExternalData].[dbo].[UtilityImport] AS UtilImp ON EGOS.UtilityImportId = UtilImp.Id
  JOIN NewClientInfo.dbo.Accounts AS A ON A.AccountID = EGOS.AccountID
  JOIN NewClientInfo.dbo.Client AS C ON C.ClientID = A.AccountClientID
  JOIN [ExternalData].[dbo].[UtilityImportRawData] AS rd ON rd.UtilityImportId = UtilImp.Id
  JOIN [IntervalData].[dbo].[AMIImportHeader] AS ami_head ON ami_head.AccountId = A.AccountID
  JOIN [IntervalData].[dbo].[AMIImport] AS ami ON ami.AMIImportHeaderId = ami_head.Id AND ami.StartDate = EGOS.DateFrom and ami.EndDate = EGOS.DateTo
  WHERE AccountUtilityProviderID = 1
  AND ami.Usage > 0
"""
df = pd.read_sql(sql1,conn1)

C:\Users\local_nmilinkovich\Temp\ipykernel_10812\4058066216.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql1,conn1)


In [8]:
calcs = \
pd.read_excel(r"Q:\AUDITORS\EGOS Shared Folder\Development\Nick's pythons\SC9 IV Calcs 5-20-2025\Output\Standard\Standard Calcs561.xlsx")

In [9]:
calcs = calcs[calcs['total'] > 0]

In [10]:
calcs['DateFrom'] = pd.to_datetime(calcs['DateFrom'])
calcs['DateTo'] = pd.to_datetime(calcs['DateTo'])

In [11]:
calcs = calcs.merge(df[['AccountID', 'RawData', 'DateFrom', 'DateTo', 'Usage', 'Demand', 'Usage Disc']], on =['AccountID', 'DateFrom', 'DateTo'])

In [12]:
calcs = calcs[(calcs['Usage Disc'] > .9) &
      (calcs['Usage Disc'] < 1.1)]

In [13]:
calcs['Days'] = (calcs['DateTo'] - calcs['DateFrom']).dt.days
calcs['Load Factor'] = calcs['Usage'] / (calcs['Demand'] * calcs['Days'] * 24)

In [14]:
calcs['AsBilled'] = calcs['RawData'].str.split('Total electricity delivery charges\n').str[1].str.split('\n').str[0].str.replace('$','').str.replace(',','').astype(float)

In [15]:
calcs['Contract Demand'] = calcs['contract_demand_charge'].apply(string_to_list).str['kW']

In [16]:
df = calcs[['AccountID', 'DateFrom', 'DateTo', 'Usage', 'Demand',
       'AsBilled', 'Usage Disc', 'Contract Demand', 'total']]

In [17]:
df['Year'] = df['DateTo'].dt.year

C:\Users\local_nmilinkovich\Temp\ipykernel_10812\2025172405.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Year'] = df['DateTo'].dt.year


In [18]:
df = df[df['Year'] >= 2024]

In [19]:
df.drop_duplicates(subset=['AccountID' ,'DateFrom', 'DateTo'], inplace=True)

In [20]:
df['Diff'] = df['AsBilled'] - df['total']

In [21]:
df[['AccountID', 'DateFrom', 'DateTo', 'Usage', 'Usage Disc', 'Contract Demand', 'AsBilled', 'total', 'Diff']]

,AccountID,DateFrom,DateTo,Usage,Usage Disc,Contract Demand,AsBilled,total,Diff
3,39241,2024-01-16,2024-02-14,134658.0,1.000002,494.4,15538.89,11880.13,3658.76
4,39241,2024-02-14,2024-03-15,129372.0,1.000004,494.4,15626.68,13164.09,2462.59
5,39241,2024-03-15,2024-04-15,129464.0,0.992441,494.4,16172.66,12087.76,4084.90
6,39241,2024-04-15,2024-05-14,127051.0,0.987721,494.4,18674.82,13017.87,5656.95
7,39241,2024-05-14,2024-06-13,174782.0,1.000005,535.6,25316.61,24012.40,1304.21
...,...,...,...,...,...,...,...,...,...
6709,65838,2024-08-14,2024-09-13,336794.0,0.996676,1243.3,63459.60,68121.42,-4661.82
6710,65838,2024-10-11,2024-11-13,302546.0,0.977117,1243.3,30564.86,32737.15,-2172.29
6713,65838,2025-01-16,2025-02-18,310228.0,0.966265,1243.3,36894.87,33175.80,3719.07
6714,65838,2025-02-18,2025-03-19,262481.0,0.977484,1243.3,29855.66,28736.43,1119.23


In [22]:
df.drop(columns = 'Year', inplace=True)

In [23]:
acc_list = df.groupby('AccountID')['Diff'].sum().reset_index().sort_values(by='Diff', ascending=False)['AccountID'].tolist()

incomplete_final_df = pd.DataFrame()
final_df = pd.DataFrame()

for acc in acc_list:
    sub_df = df[df['AccountID'] == acc].sort_values(by='DateTo')
    if len(sub_df) > 12:
        sub_df = sub_df.tail(12)

    sub_df = sub_df[['AccountID','DateFrom','DateTo','Usage','Usage Disc','Demand','Contract Demand', 'AsBilled','total','Diff']]
    
    as_billed = sub_df['AsBilled'].sum()
    Standby = sub_df['total'].sum()
    total_savings = sub_df['Diff'].sum()

    cols_len = len(sub_df.columns.tolist())

    row_plus_one = (cols_len - 3) * [' '] + [old_rate, new_rate, 'Savings']
    row_plus_two = (cols_len - 3) * [' '] + [as_billed, Standby, total_savings]
    row_plus_three = cols_len * [' ']

    sub_df.loc[len(sub_df)] = row_plus_one
    sub_df.loc[len(sub_df)] = row_plus_two
    sub_df.loc[len(sub_df)] = row_plus_three

    if len(sub_df) >= 15:
        final_df = pd.concat([final_df, sub_df])

    incomplete_final_df = pd.concat([incomplete_final_df, sub_df])

    progress = acc_list.index(acc)
    left = len(acc_list) - progress
    print(f'[{progress * "="}{left * "-"}]')
    clear_output(wait=True)
    
print(f'[{len(acc_list) * "="}]')

[===================================================================================================================================================================================================================================================================================================================================================================================================================================================]


In [24]:
incomplete_final_df = incomplete_final_df.merge(Acc(incomplete_final_df[incomplete_final_df['AccountID'] != ' ']['AccountID'].dropna().astype(str).tolist(), conn1), how='left')
final_df = final_df.merge(Acc(final_df[final_df['AccountID'] != ' ']['AccountID'].dropna().astype(str).tolist(), conn1), how='left')

C:\Users\local_nmilinkovich\Temp\ipykernel_10812\4109330817.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  bills = pd.read_sql(sql1, conn1)
C:\Users\local_nmilinkovich\Temp\ipykernel_10812\4109330817.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  bills = pd.read_sql(sql1, conn1)


In [25]:
incomplete_final_df = incomplete_final_df[['AccountNumber','Client', 'AccountID', 'DateFrom', 'DateTo', 'Usage', 'Demand', 'Usage Disc', 'Contract Demand', 'AsBilled',
        'total', 'Diff']].rename(columns = {
           'Usage Disc':'AMI / Bill Usage',
           'AsBilled':old_rate,
           'total':new_rate,
           'Diff':'Savings'
       })

In [26]:
final_df = final_df[['AccountNumber','Client', 'AccountID', 'DateFrom', 'DateTo', 'Usage', 'Demand', 'Usage Disc', 'Contract Demand', 'AsBilled',
        'total', 'Diff']].rename(columns = {
           'Usage Disc':'AMI / Bill Usage',
           'AsBilled':old_rate,
           'total':new_rate,
           'Diff':'Savings'
       })

In [27]:
incomplete_final_df.to_excel(f'H:\Downloads\{new_rate} Standby Analysis Partial Data {datetime.now().strftime("%m-%d-%Y")}.xlsx', index=False)
final_df.to_excel(f'H:\Downloads\{new_rate} Standby Analysis {datetime.now().strftime("%m-%d-%Y")}.xlsx', index=False)

END CLEANING

In [28]:
diff = final_df[(final_df['Contract Demand'] == ' ') &
~(final_df['Savings'].isin(['Savings', ' ']))]['Savings'].astype(float).values

In [29]:
diff[np.where(diff < 0)].mean()

-4462.043617926788

In [30]:
diff[np.where(diff > 0)].mean()

2253.3648851694506

In [31]:
len(diff[np.where(diff < 0)])

130

In [32]:
final_df['AccountID'].nunique()

252

In [33]:
(final_df[(final_df['AccountNumber'].astype(str).str.len() < 5) &
         (~final_df['Savings'].isin(['Savings', ' ']))]['Savings'] > 0).value_counts()

Savings
False    130
True     121
Name: count, dtype: int64

In [93]:
final_df.dropna().groupby('AccountID')['Savings'].sum().reset_index()

,AccountID,Savings
0,20845,945.885868
1,20856,-25228.111561
2,23362,100.81
3,23364,211.82
4,23549,-13533.65
...,...,...
246,65827,889.230132
247,65836,6700.55
248,65838,-14315.98
249,65842,-1959.06


In [109]:
proto = final_df.dropna().groupby('AccountID')['Savings'].sum().reset_index()
saved = final_df[final_df['AccountID'].isin(proto[proto['Savings'] > 0]['AccountID'].tolist())]
lost = final_df[final_df['AccountID'].isin(proto[proto['Savings'] < 0]['AccountID'].tolist())]

In [110]:
print(f"Customers who saved saving pct: {(saved.groupby('AccountID')['Savings'].sum() / saved.groupby('AccountID')[old_rate].sum()).mean().round(3) * 100}%")
print(f"Customers who saved savings: ${(saved.groupby('AccountID')['Savings'].sum().mean().round(2))}")

print(f"Customers who saved saving pct: {(lost.groupby('AccountID')['Savings'].sum() / lost.groupby('AccountID')[old_rate].sum()).mean().round(3) * 100}%")
print(f"Customers who saved savings: ${(lost.groupby('AccountID')['Savings'].sum().mean().round(2))}")

Customers who saved saving pct: 7.1%
Customers who saved savings: $2253.36
Customers who saved saving pct: -19.6%
Customers who saved savings: $-4462.04


In [112]:
saved['Savings'].sum()

272657.1511055037

PRINTING CHARGES FOR VERIFICATION

In [36]:
demo = calcs[calcs['DateFrom'] == '1/16/2024'].iloc[1][calcs.columns[:6]].reset_index().T.to_dict()

In [91]:
demo[0]

{'index': 'customer_charge',
 355: "{'amount': 63.8, 'description': 'Fixed monthly customer service charge'}"}

In [39]:
final_df['AccountID'].nunique()

252

In [38]:
incomplete_final_df['AccountID'].nunique()

436

In [35]:
import pprint

def pretty_print_charges(data):
    for key, value in data.items():
        print(f"\n=== Component {key + 1} ===")
        print(f"Type: {value.get('index')}")
        
        for inner_key, inner_val in value.items():
            if inner_key == 'index':
                continue
            try:
                parsed = ast.literal_eval(inner_val)
                for k, v in parsed.items():
                    if k == "breakdown" and isinstance(v, dict):
                        print("Breakdown:")
                        for subk, subv in v.items():
                            print(f"  - {subk.replace('_', ' ').capitalize()}:")
                            for leafk, leafv in subv.items():
                                print(f"      {leafk.replace('_', ' ')}: {leafv}")
                    else:
                        print(f"{k.replace('_', ' ').capitalize()}: {v}")
            except (ValueError, SyntaxError):
                print(f"Raw Value: {inner_val}")



In [37]:
pretty_print_charges(demo)


=== Component 1 ===
Type: customer_charge
Amount: 63.8
Description: Fixed monthly customer service charge

=== Component 2 ===
Type: processing_charge
Amount: 0.0
Description: Billing & payment processing fee

=== Component 3 ===
Type: energy_surcharge
Amount: 79.29
Rate per kwh: 0.004864
Usage kwh: 16302.0
Breakdown:
  - Mac reconciliation:
      rate per kWh: 0.001985
      charge: 32.35947
  - Mac uncollectiblebill expense:
      rate per kWh: 0.00026
      charge: 4.23852
  - Mac transition adjustment:
      rate per kWh: 7e-06
      charge: 0.114114
  - Monthly adjustment clause:
      rate per kWh: 0.00384
      charge: 62.59968
  - Revenue decoupling mechanism adjustment:
      rate per kWh: -0.0015
      charge: -24.453
  - Clean energy fund surcharge:
      rate per kWh: 0.0
      charge: 0.0
  - Energy efficiency tracker:
      rate per kWh: 0.0
      charge: 0.0
  - Ancillary services charges:
      rate per kWh: 0.0
      charge: 0.0
  - Nypa transmission adjustment charge